In [ ]:
#Ce programme permet d'entrainer notre model sur notre propre dataset
#Notre model sera basé sur un modele CNN VGG16

In [1]:
#!pip install scikit-image

In [2]:
#!pip install SciPy

In [3]:
#!pip install h5py

In [4]:
import scipy
import tensorflow as tf

In [8]:
import numpy as np
import pandas as pd
from glob import glob
import os
 
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
 
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow

In [ ]:
#Définition de la taille de l'image
#src_path_train = "input"   => Indiquer le nom du dossier contenant le dataset d'entrainement
#src_path_test = "input test"  => Indiquer le nom du dossier contenant le dataset de test


In [9]:
IMSIZE=[100,100]

src_path_train = "input"
src_path_test = "input test"
batch_size = 32

In [ ]:
#Récuperation du dataset de train et de test

In [10]:

 
image_gen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

# create generators
train_generator = image_gen.flow_from_directory(
  src_path_train,
  target_size=IMSIZE,
  shuffle=True,
  batch_size=batch_size,
)
 
test_generator = image_gen.flow_from_directory(
  src_path_test,
  target_size=IMSIZE,
  shuffle=True,
  batch_size=batch_size,
)

Found 5996 images belonging to 2 classes.
Found 638 images belonging to 2 classes.


In [11]:
from glob import glob
train_image_files = glob(src_path_train + '/*/*.jp*g')
test_image_files = glob(src_path_test + '/*/*.jp*g')
len(train_image_files), len(test_image_files)
import numpy as np 
import pandas as pd




In [12]:
len(train_generator)

188

In [13]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1])

In [14]:
len(train_image_files)

5996

In [ ]:
#indiquer le nombres de classes dans notre dataset (le nombre de sous-dossier dans le dossier de train et de test)

In [15]:
NBCLASSES = 2

In [10]:
def create_model():
    vgg = VGG16(input_shape=IMSIZE + [3], weights='imagenet', include_top=False)

    # Freeze existing VGG already trained weights
    for layer in vgg.layers:
        layer.trainable = False
    
    # get the VGG output
    out = vgg.output
    
    # Add new dense layer at the end
    x = Flatten()(out)
    x = Dense(NBCLASSES, activation='softmax')(x)
    
    model = Model(inputs=vgg.input, outputs=x)
    
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])
    
    model.summary()
    
    return model

mymodel = create_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [12]:
mymodel.fit(train_generator,epochs=30,batch_size=32)

Epoch 1/30
188/188 [==============================] - 177s 938ms/step - loss: 0.1895 - accuracy: 0.9335
Epoch 2/30
188/188 [==============================] - 164s 872ms/step - loss: 0.1158 - accuracy: 0.9586
Epoch 3/30
188/188 [==============================] - 164s 872ms/step - loss: 0.1032 - accuracy: 0.9611
Epoch 4/30
188/188 [==============================] - 164s 871ms/step - loss: 0.0926 - accuracy: 0.9648
Epoch 5/30
188/188 [==============================] - 164s 870ms/step - loss: 0.0840 - accuracy: 0.9690
Epoch 6/30
188/188 [==============================] - 164s 871ms/step - loss: 0.0817 - accuracy: 0.9705
Epoch 7/30
188/188 [==============================] - 165s 875ms/step - loss: 0.0751 - accuracy: 0.9720
Epoch 8/30
188/188 [==============================] - 165s 876ms/step - loss: 0.0753 - accuracy: 0.9733
Epoch 9/30
188/188 [==============================] - 165s 875ms/step - loss: 0.0699 - accuracy: 0.9741
Epoch 10/30
188/188 [==============================] - 166s 884m

In [13]:
score = mymodel.evaluate(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 22s 1s/step - loss: 0.0803 - accuracy: 0.9671
Test loss: 0.08031278848648071
Test accuracy: 0.9670846462249756


In [ ]:
#Sauvegarde su model

In [14]:
mymodel.save('CNN_doc_scientifique.h5')

In [16]:
mymodel=tf.keras.models.load_model('CNN_doc_scientifique.h5')

In [ ]:
# fonction pour mettre un nom sur l'image en fonction de la catégorie

In [ ]:
#
#
#Il est conseillé de basculer sur le bloc note 4 pour faire la prediction
#
#

In [ ]:
#La suite du programme est pour interpreter la prediction (donne la catégorie)
#Attention le code est basé sur 4 catégories à changer!!

In [17]:
def labelisemonimage(nom_image):
    image = tf.keras.utils.load_img(nom_image, target_size=(100, 100))
    image = tf.keras.utils.img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    prediction = mymodel.predict(image)
    #print(prediction)
    if (np.argmax(prediction)==0):
        print("cat 0 image extraite d'un Journal  precision de :" ,prediction[0,0]*100,"%")
    elif (np.argmax(prediction)==1):
        print("cat 1 image extraite d'un Magazine scientifique precision de :" ,prediction[0,1]*100,"%")
    elif (np.argmax(prediction)==2):
        print("cat 2  image extraite d'une Documentation scientifique precision de :" ,prediction[0,2]*100,"%")
    elif (np.argmax(prediction)==3):
        print("cat 3 logo de l'US air force avec % de :" ,prediction[0,3]*100,"%")


In [18]:
def labelisemonimage2(nom_image):
    image = tf.keras.utils.load_img(nom_image, target_size=(100, 100))
    image = tf.keras.utils.img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    prediction = mymodel.predict(image,verbose=0)
    return(np.argmax(prediction))

In [19]:
#labelisemonimage('marine2.jpg')

In [20]:
#labelisemonimage('JPG prevision/micromachines-13-01593page14.jpg')

In [21]:
dossier='JPG prevision'

In [22]:
from os import listdir
from os.path import isfile, join
fichiers = [f for f in listdir(dossier) if isfile(join(dossier, f))]

In [23]:
#fichiers

In [24]:

#for doc in fichiers:
       #labelisemonimage(dossier+'/'+doc)

In [25]:
#prediction=mymodel.predict('JPG prevision/images.jpg')

In [26]:
listedocument=[]
result=0
resultlabel=0
listeresult=[]
nb_page=-1

nom_prem_document=fichiers[0].split('page',1)
nom_prem_document[0]
listedocument.append(nom_prem_document[0])
for doc in fichiers:
    nom_document=doc.split('page')[0]
    resultlabel=labelisemonimage2(dossier+'/'+doc)
    nb_page=nb_page+1
    
   
    if nom_document!=listedocument[-1]:
        #print (result)
        #print (nb_page)
        result=int(result)/(nb_page)

        listeresult.append(result)
        listedocument.append(nom_document)
        result=resultlabel
        result=resultlabel
        nb_page=0
    else :
        result=result+resultlabel
listeresult.append(result/nb_page)  
    

In [27]:
listedocument

['fichier10',
 'fichier12',
 'fichier13',
 'fichier14',
 'fichier1',
 'fichier2',
 'fichier3',
 'fichier4',
 'fichier5',
 'fichier6',
 'fichier7',
 'fichier8']

In [28]:
listeresult

[0.5,
 0.14285714285714285,
 0.7692307692307693,
 0.6666666666666666,
 0.7816901408450704,
 0.058823529411764705,
 0.14285714285714285,
 0.46875,
 0.8279569892473119,
 0.75,
 0.9436619718309859,
 0.6666666666666666]

In [29]:
i=0
for doc in listedocument:
    if listeresult[i]>0.5:
        res='Doc Scientifique'
        proba=listeresult[i]*100
    else:
        res="pdf sans interet"
        proba=(1-listeresult[i])*100
    print("Le "+doc+" est un "+res+" probabilité de : "+str(proba)+" %")
    i=i+1

Le fichier10 est un pdf sans interet probabilité de : 50.0 %
Le fichier12 est un pdf sans interet probabilité de : 85.71428571428572 %
Le fichier13 est un Doc Scientifique probabilité de : 76.92307692307693 %
Le fichier14 est un Doc Scientifique probabilité de : 66.66666666666666 %
Le fichier1 est un Doc Scientifique probabilité de : 78.16901408450704 %
Le fichier2 est un pdf sans interet probabilité de : 94.11764705882352 %
Le fichier3 est un pdf sans interet probabilité de : 85.71428571428572 %
Le fichier4 est un pdf sans interet probabilité de : 53.125 %
Le fichier5 est un Doc Scientifique probabilité de : 82.79569892473118 %
Le fichier6 est un Doc Scientifique probabilité de : 75.0 %
Le fichier7 est un Doc Scientifique probabilité de : 94.36619718309859 %
Le fichier8 est un Doc Scientifique probabilité de : 66.66666666666666 %


In [30]:
#vide le dossier telechargement PDF
#vide le dossier JPG prevision